In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import zipfile
%matplotlib notebook

In [2]:
# path = r"C:\Users\parik\Documents\Data sets\Consumption Pyramids-20220311T061932Z-001\Consumption Pyramids\Monthly Expenses"
# all_files = glob.glob(path + "/*.csv")
# files2019 = []
# for path in all_files:
#     if "2019" in path:
#         files2019.append(path)
        
# files2019 = files2019[-4:-1]

# li = []

# for filename in files2019:
#     df = pd.read_csv(filename, index_col=None, header=0)
#     li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)
# df_merged = pd.read_csv('df_merged_ex.csv')
# df_merged.drop('Unnamed: 0',axis = 1,inplace=True)
# merged_exps = pd.merge(misc_exp,df_merged, how = "inner", on = "HH_ID")
# bride = merged_exps['is_bride']
# groom = ~bride

Make a tuple of files to merge.

In [3]:
aspiration_path = r"C:\Users\parik\Documents\Data sets\aspirations\drive-download-20220221T055656Z-001"
aspiration_files = glob.glob(aspiration_path + "/*.csv")
aspiration_files = aspiration_files[16:]
income_path = r"C:\Users\parik\Documents\Data sets\income pyramids\Member Incomes"
income_files = glob.glob(income_path + "/*.csv")
income_files = income_files[60:]
files_to_keep = []
for x in range(len(income_files)):
    if x%4 == 0:
        files_to_keep.append(income_files[x])
income_files = files_to_keep

exp_path = r"C:\Users\parik\Documents\Data sets\Consumption Pyramids-20220311T061932Z-001\Consumption Pyramids\Monthly Expenses"
exp_files = glob.glob(exp_path + "/*.csv")
exp_files = exp_files[60:]

list_exp_paths = []
for x in range(len(exp_files)//3):
    file1 = exp_files[3*x]
    file2 = exp_files[3*x + 1]
    file3 = exp_files[3*x + 2]
    files = file1,file2,file3
    list_exp_paths.append(files)
exp_files = list_exp_paths[1:-1]
    

Steps to move ahead:

1. open aspirations
2. hhid of households that borrowed for wedding at sometime.
3. open income 
4. Identify member id, age, gender of member.
5. open cphs
6. merge hhid,memid,age,gender with cphs

In [4]:
bi = []
for x in range(len(income_files)):
    df = pd.read_csv(aspiration_files[x])
    cols_to_keep = ['WAVE_NO', 'HH_ID', 'AGE_GROUP',
     'INCOME_GROUP',
     'OCCUPATION_GROUP',
     'EDUCATION_GROUP',
     'GENDER_GROUP',
     'WATER_GROUP',
     'POWER_GROUP',
     'TRAVEL_GROUP',
     'SIZE_GROUP',
    'BORROWED_FOR_WEDDING']
    df = df[cols_to_keep]
    df = df[df['BORROWED_FOR_WEDDING'] == 'Y']
    df2 = pd.read_csv(income_files[x])
    df2cols = ['HH_ID', 'MEM_ID','GENDER', 'AGE_YRS','AGE_MTHS','MARITAL_STATUS','LITERACY','EDUCATION','INCOME_OF_MEMBER_FROM_ALL_SOURCES']
    df2 = df2[df2cols]
    df2['unmarried'] = df2['MARITAL_STATUS'] == "Unmarried"
    df2 = df2[df2['unmarried']]
    df2_max_gender_of_unmarried  = df2.loc[df2.groupby(["HH_ID"])["AGE_MTHS"].idxmax()] 
    df2_max_gender_of_unmarried.rename(columns={'GENDER':'gender_married', "AGE_YRS": 'age_married'}, inplace=True)
    df_merged = pd.merge(df,df2_max_gender_of_unmarried, how = "inner", on = "HH_ID")
    
    li = []
    for filename in exp_files[x]:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)
    frame = pd.concat(li, axis=0, ignore_index=True)
    misc_exp = frame.groupby('HH_ID').agg(np.sum)
    merged_exps = pd.merge(misc_exp,df_merged, how = "inner", on = "HH_ID")
    cols_to_keep = ['HH_ID', 
                    'WAVE_NO',
     'HH_WEIGHT_MS',
     'HH_WEIGHT_FOR_COUNTRY_MS',
     'HH_WEIGHT_FOR_STATE_MS',
     'HH_NON_RESPONSE_MS',
     'HH_NON_RESPONSE_FOR_COUNTRY_MS',
     'HH_NON_RESPONSE_FOR_STATE_MS',
     'TOTAL_EXPENDITURE',
    'MONTHLY_EXPENSE_ON_MISCELLANEOUS',
     'ADJ_TOTAL_EXPENDITURE',
    'WAVE_NO',
     'AGE_GROUP',
     'INCOME_GROUP',
     'OCCUPATION_GROUP',
     'EDUCATION_GROUP',
     'GENDER_GROUP',
     'WATER_GROUP',
     'POWER_GROUP',
     'TRAVEL_GROUP',
     'SIZE_GROUP',
     'BORROWED_FOR_WEDDING',
     'MEM_ID',
     'gender_married',
     'age_married',
     'AGE_MTHS',
     'MARITAL_STATUS',
     'LITERACY',
     'EDUCATION',
     'INCOME_OF_MEMBER_FROM_ALL_SOURCES',
     'unmarried']
    merged_exps = merged_exps[cols_to_keep]
    bi.append(merged_exps)

In [5]:
frame = pd.concat(bi, axis=0, ignore_index=True)

In [6]:
frame.shape

(4901, 31)

In [7]:
frame.columns

Index(['HH_ID', 'WAVE_NO', 'HH_WEIGHT_MS', 'HH_WEIGHT_FOR_COUNTRY_MS',
       'HH_WEIGHT_FOR_STATE_MS', 'HH_NON_RESPONSE_MS',
       'HH_NON_RESPONSE_FOR_COUNTRY_MS', 'HH_NON_RESPONSE_FOR_STATE_MS',
       'TOTAL_EXPENDITURE', 'MONTHLY_EXPENSE_ON_MISCELLANEOUS',
       'ADJ_TOTAL_EXPENDITURE', 'WAVE_NO', 'AGE_GROUP', 'INCOME_GROUP',
       'OCCUPATION_GROUP', 'EDUCATION_GROUP', 'GENDER_GROUP', 'WATER_GROUP',
       'POWER_GROUP', 'TRAVEL_GROUP', 'SIZE_GROUP', 'BORROWED_FOR_WEDDING',
       'MEM_ID', 'gender_married', 'age_married', 'AGE_MTHS', 'MARITAL_STATUS',
       'LITERACY', 'EDUCATION', 'INCOME_OF_MEMBER_FROM_ALL_SOURCES',
       'unmarried'],
      dtype='object')

In [8]:
bride = merged_exps['gender_married'] == 'F'
groom = ~bride

In [11]:
merged_exps[bride]["TOTAL_EXPENDITURE"].mean()

27786.648780487805

In [12]:
merged_exps[groom]["TOTAL_EXPENDITURE"].mean()

31166.83816793893